In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import os
import requests
import io
import PIL
import PIL.Image
import math
import pathlib
import numpy as np

In [2]:
batch_size = 8 # 每⼀批所处理的图⽚数量
img_height = 256 # 图⽚⾼度，单位为像素
img_width = 256 # 图⽚宽度，单位为像素

TF_SERVING_BASE_URL = 'http://localhost:8501/'
image_dir = './datasets/flower_photos'

task_id = '350002'
task_dir = os.path.join('./datasets/', task_id)
train_dir = os.path.join(task_dir, 'train')
test_dir = os.path.join(task_dir, 'test')

In [3]:
import shutil

id_code_mapping = { # 根据web界⾯显示情况设定
'daisy' : '0',
'dandelion' : '1', 
'roses' : '2', 
'sunflowers' : '3', 
'tulips' : '4', 
}
total_image_count = len(list(pathlib.Path(image_dir).glob('*/*.jpg')))
print(total_image_count)
print(train_dir)

for img in pathlib.Path(image_dir).glob('*/*.jpg'): 
    if np.random.ranf() < 0.8: # 需要通过修改数值来控制测试集的⽐例
      dst_dir = os.path.join(train_dir, os.path.basename(os.path.dirname(img)))
    else:
      dst_dir = os.path.join(test_dir, os.path.basename(os.path.dirname(img)))
    if not os.path.exists(dst_dir):
      os.makedirs(dst_dir)
    #print(os.path.dirname(img))
    dst = os.path.join(dst_dir, img.name)
    #print(dst)
    #dst = os.path.join(dst_dir, record['id']+'.png')
    #img.save(dst)
    shutil.copy(img, dst)

3652
./datasets/350002/train


In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
train_dir,
validation_split=0.2, # 设定验证集⽐例
subset="training",
seed=123,
#image_size=(img_height, img_width),
batch_size=batch_size)

Found 3723 files belonging to 7 classes.
Using 2979 files for training.


In [5]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
train_dir,
validation_split=0.2, # 设定验证集⽐例
subset="validation",
seed=123,
#image_size=(img_height, img_width),
batch_size=batch_size)

Found 3723 files belonging to 7 classes.
Using 744 files for validation.


In [6]:
print(train_ds.class_names)
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

['0', '1', 'daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
(8, 256, 256, 3)
(8,)


In [7]:
augmentation_dict = {
    'RandomFlip': tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    'RandomRotation': tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    'RandomContrast': tf.keras.layers.experimental.preprocessing.RandomContrast(0.2),
    'RandomZoom': tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=0.1, width_factor=0.1),
    'RandomTranslation': tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
    'RandomCrop': tf.keras.layers.experimental.preprocessing.RandomCrop(img_height, img_width),

}

In [8]:
num_classes = 5
model = tf.keras.Sequential([ # 根据需要调整模型结构
layers.experimental.preprocessing.Resizing(img_height, img_width),
tf.keras.layers.experimental.preprocessing.Rescaling(1. / 255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
   
])

In [9]:
model.compile(
optimizer='adam',
loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=['accuracy'])

In [10]:
# 定义TensorBoard对象.histogram_freq 如果设置为0，则不会计算直方图。
log_dir = os.path.join("logs")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)  

model.fit(
train_ds,
validation_data=val_ds,
epochs=5, callbacks=[tensorboard_callback]
)

Epoch 1/5


InvalidArgumentError:  Received a label value of 6 which is outside the valid range of [0, 5).  Label values: 6 6 5 2 2 3 4 2
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-10-864eb9a8248b>:5) ]] [Op:__inference_train_function_901]

Function call stack:
train_function


In [ ]:
model.summary()

In [ ]:
tf.keras.models.save_model(
model,
'./models/image/1/', # ./models为tensorflow-serving的模型根⽬录
overwrite=True,
include_optimizer=True,
save_format=None,
signatures=None,
options=None
)

In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
def test_image_model(test_dir,  code, batch_size=10): 
  imgs = []
  codes = []
  imgdir = os.path.join(test_dir, val_ds.class_names[code]) 
  print(imgdir)
  right = 0
  wrong = 0
  for i in pathlib.Path(imgdir).glob('./*.jpg'): 
    #print(i.name)
    imgs = []
    img = PIL.Image.open(i)
    pixels = np.array(img)
    plt.imshow(pixels)
    imgs.append(pixels.tolist()) 
    img.close()
    for j in range(int(math.ceil(len(imgs)/batch_size))):
      req_data = json.dumps({
        'inputs': imgs[j*batch_size:(j+1)*batch_size],
      })
    
    #data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:3].tolist()})
    #http://localhost:8501/v1/models/fashion_mnist:predict
    #print(TF_SERVING_BASE_URL+'v1/models/image/versions/1:predict')
    #response = requests.post(TF_SERVING_BASE_URL+'v1/models/image/versions/1:predict', # 根据部署地址填写
    #json_response = requests.post('http://localhost:8501/v1/models/fashion_mnist:predict', data=data, headers=headers)
    response = requests.post(TF_SERVING_BASE_URL+'v1/models/image:predict', # 根据部署地址填写
    data=req_data,
    headers={"content-type": "application/json"})
    #print(response.status_code)
    if response.status_code != 200:
      raise RuntimeError('Request tf-serving failed: ' + response.text)
    resp_data = json.loads(response.text) 
    if 'outputs' not in resp_data \
         or type(resp_data['outputs']) is not list:
      raise ValueError('Malformed tf-serving response')
    #print(np.argmax(resp_data['outputs']))
    if( int(np.argmax(resp_data['outputs']) == int(code) )):
       right = right + 1
    else:
       wrong = wrong + 1
    #codes.extend(np.argmax(resp_data['outputs'], axis=1).tolist())
    #print("class:"+ str(np.argmax(resp_data['outputs'])))
  return right/(right+wrong)

In [ ]:
codes = test_image_model(test_dir,  0)
print('类别0的准确率', codes)
codes = test_image_model(test_dir, 1)
print('类别1的准确率', codes)